In [ ]:
"""
Example script to train a network to generate text with the style of a given corpus
--By word--
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
Based on
https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py
20 epochs should be enough to get decent results.
Uses data generator to avoid loading all the test set into memory.
Saves the weights and model every epoch.
"""

'\nExample script to train a network to generate text with the style of a given corpus\n--By word--\nIt is recommended to run this script on GPU, as recurrent\nnetworks are quite computationally intensive.\nBased on\nhttps://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py\n20 epochs should be enough to get decent results.\nUses data generator to avoid loading all the test set into memory.\nSaves the weights and model every epoch.\n'

In [ ]:
from __future__ import print_function
from keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional
import numpy as np
import sys
import io
import os
import codecs

In [ ]:
# Parameters: change to experiment different configurations
SEQUENCE_LEN = 10
MIN_WORD_FREQUENCY = 10
STEP = 1
BATCH_SIZE = 32

In [ ]:
def shuffle_and_split_training_set(sentences_original, next_original, percentage_test=2):
    # shuffle at unison
    print('Shuffling sentences')

    tmp_sentences = []
    tmp_next_word = []
    for i in np.random.permutation(len(sentences_original)):
        tmp_sentences.append(sentences_original[i])
        tmp_next_word.append(next_original[i])

    cut_index = int(len(sentences_original) * (1.-(percentage_test/100.)))
    x_train, x_test = tmp_sentences[:cut_index], tmp_sentences[cut_index:]
    y_train, y_test = tmp_next_word[:cut_index], tmp_next_word[cut_index:]

    print("Size of training set = %d" % len(x_train))
    print("Size of test set = %d" % len(y_test))
    return (x_train, y_train), (x_test, y_test)


# Data generator for fit and evaluate
def generator(sentence_list, next_word_list, batch_size):
    index = 0
    while True:
        x = np.zeros((batch_size, SEQUENCE_LEN, len(words)), dtype=bool)
        y = np.zeros((batch_size, len(words)), dtype=bool)
        for i in range(batch_size):
            for t, w in enumerate(sentence_list[index % len(sentence_list)]):
                x[i, t, word_indices[w]] = 1
            y[i, word_indices[next_word_list[index % len(sentence_list)]]] = 1
            index = index + 1
        yield x, y


def print_vocabulary(words_file_path, words_set):
    words_file = codecs.open(words_file_path, 'w', encoding='utf8')
    for w in words_set:
        if w != "\n":
            words_file.write(w+"\n")
        else:
            words_file.write(w)
    words_file.close()


def get_model(dropout=0.2):
    print('Build model...')
    model = Sequential()
    model.add(Bidirectional(LSTM(128), input_shape=(SEQUENCE_LEN, len(words))))
    if dropout > 0:
        model.add(Dropout(dropout))
    model.add(Dense(len(words)))
    model.add(Activation('softmax'))
    return model


# Functions from keras-team/keras/blob/master/examples/lstm_text_generation.py
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    examples_file.write('\n----- Generating text after Epoch: %d\n' % epoch)

    # Randomly pick a seed sequence
    seed_index = np.random.randint(len(sentences+sentences_test))
    seed = (sentences+sentences_test)[seed_index]

    for diversity in [0.3, 0.4, 0.5, 0.6, 0.7]:
        sentence = seed
        examples_file.write('----- Diversity:' + str(diversity) + '\n')
        examples_file.write('----- Generating with seed:\n"' + ' '.join(sentence) + '"\n')
        examples_file.write(' '.join(sentence))

        for i in range(50):
            x_pred = np.zeros((1, SEQUENCE_LEN, len(words)))
            for t, word in enumerate(sentence):
                x_pred[0, t, word_indices[word]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]

            sentence = sentence[1:]
            sentence.append(next_word)

            examples_file.write(" "+next_word)
        examples_file.write('\n')
    examples_file.write('='*80 + '\n')
    examples_file.flush()


In [ ]:
with open('../content/drive/MyDrive/Data/select_lyrics_data.txt', encoding='utf-8') as f:
    text = f.read().lower().replace('\n', ' \n ')
print('Corpus length in characters:', len(text))

text_in_words = [w for w in text.split(' ') if w.strip() != '' or w == '\n']
print('Corpus length in words:', len(text_in_words))

# Calculate word frequency
word_freq = {}
for word in text_in_words:
    word_freq[word] = word_freq.get(word, 0) + 1

ignored_words = set()
for k, v in word_freq.items():
    if word_freq[k] < MIN_WORD_FREQUENCY:
        ignored_words.add(k)

words = set(text_in_words)
print('Unique words before ignoring:', len(words))
print('Ignoring words with frequency <', MIN_WORD_FREQUENCY)
words = sorted(set(words) - ignored_words)
print('Unique words after ignoring:', len(words))
print_vocabulary('../content/drive/MyDrive/Data/vocab_v4.txt', words)

word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))

# cut the text in semi-redundant sequences of SEQUENCE_LEN words
sentences = []
next_words = []
ignored = 0
for i in range(0, len(text_in_words) - SEQUENCE_LEN, STEP):
    # Only add the sequences where no word is in ignored_words
    if len(set(text_in_words[i: i+SEQUENCE_LEN+1]).intersection(ignored_words)) == 0:
        sentences.append(text_in_words[i: i + SEQUENCE_LEN])
        next_words.append(text_in_words[i + SEQUENCE_LEN])
    else:
        ignored = ignored + 1
print('Ignored sequences:', ignored)
print('Remaining sequences:', len(sentences))

Corpus length in characters: 72959776
Corpus length in words: 15801923
Unique words before ignoring: 248649
Ignoring words with frequency < 10
Unique words after ignoring: 28939
Ignored sequences: 3196843
Remaining sequences: 12605070


In [ ]:
# x, y, x_test, y_test
(sentences, next_words), (sentences_test, next_words_test) = shuffle_and_split_training_set(
    sentences[:500000], next_words[:500000]
)

Shuffling sentences
Size of training set = 490000
Size of test set = 10000


In [ ]:
model = get_model()
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

file_path = "../content/drive/MyDrive/Data/Logs/LSTM_LYRICS-epoch{epoch:03d}-words%d-sequence%d-minfreq%d-" \
            "loss{loss:.4f}-acc{acc:.4f}-val_loss{val_loss:.4f}-val_acc{val_acc:.4f}" % \
            (len(words), SEQUENCE_LEN, MIN_WORD_FREQUENCY)

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', save_best_only=True)
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
early_stopping = EarlyStopping(monitor='val_acc', patience=5)
callbacks_list = [print_callback, early_stopping]

examples_file = open('../content/drive/MyDrive/Data/Logs/Example.txt', "w")
model.fit(  generator(sentences, next_words, BATCH_SIZE),
            steps_per_epoch=int(len(sentences)/BATCH_SIZE) + 1,
            epochs=25,
            callbacks=callbacks_list,
            validation_data=generator(sentences_test, next_words_test, BATCH_SIZE),
            validation_steps=int(len(sentences_test)/BATCH_SIZE) + 1)

Build model...
Epoch 1/25
15313/15313 [==============================] - ETA: 0s - loss: 5.2775 - accuracy: 0.2066

15313/15313 [==============================] - 607s 39ms/step - loss: 5.2775 - accuracy: 0.2066 - val_loss: 4.6718 - val_accuracy: 0.2541
Epoch 2/25
15313/15313 [==============================] - ETA: 0s - loss: 4.4219 - accuracy: 0.2681

15313/15313 [==============================] - 597s 39ms/step - loss: 4.4219 - accuracy: 0.2681 - val_loss: 4.3367 - val_accuracy: 0.2861
Epoch 3/25
15313/15313 [==============================] - ETA: 0s - loss: 3.9517 - accuracy: 0.3158

15313/15313 [==============================] - 597s 39ms/step - loss: 3.9517 - accuracy: 0.3158 - val_loss: 4.1607 - val_accuracy: 0.3122
Epoch 4/25
15313/15313 [==============================] - ETA: 0s - loss: 3.5599 - accuracy: 0.3640

15313/15313 [==============================] - 595s 39ms/step - loss: 3.5599 - accuracy: 0.3640 - val_loss: 4.1027 - val_accuracy: 0.3302
Epoch 5/25
15312/15313 [============================>.] - ETA: 0s - loss: 3.2122 - accuracy: 0.4101

...

15313/15313 [==============================] - 596s 39ms/step - loss: 2.6388 - accuracy: 0.4921 - val_loss: 4.2070 - val_accuracy: 0.3594
Epoch 8/25
15313/15313 [==============================] - ETA: 0s - loss: 2.4104 - accuracy: 0.5275

15313/15313 [==============================] - 595s 39ms/step - loss: 2.4104 - accuracy: 0.5275 - val_loss: 4.2506 - val_accuracy: 0.3600
Epoch 9/25
15313/15313 [==============================] - ETA: 0s - loss: 2.2087 - accuracy: 0.5591

15313/15313 [==============================] - 595s 39ms/step - loss: 2.2087 - accuracy: 0.5591 - val_loss: 4.3303 - val_accuracy: 0.3641
Epoch 10/25
15312/15313 [============================>.] - ETA: 0s - loss: 2.0398 - accuracy: 0.5870

15313/15313 [==============================] - 597s 39ms/step - loss: 2.0398 - accuracy: 0.5870 - val_loss: 4.3851 - val_accuracy: 0.3662
Epoch 11/25
15313/15313 [==============================] - ETA: 0s - loss: 1.8885 - accuracy: 0.6118

15313/15313 [==============================] - 598s 39ms/step - loss: 1.8885 - accuracy: 0.6118 - val_loss: 4.4237 - val_accuracy: 0.3701
Epoch 12/25
15100/15313 [============================>.] - ETA: 8s - loss: 1.7527 - accuracy: 0.6350

In [ ]:
model.summary()

In [ ]:
model.save('../content/drive/MyDrive/Data/LSTM_simple_model_v4.h5')